In [23]:
import urllib.request
import pandas as pd
import requests

class Crawl():
    
    def __init__(self, client_id, client_secret):

        self.client_id = client_id
        self.client_secret = client_secret
        # self.args = args
    
    def __call__(self, query, number):

        headers = {
        "X-Naver-Client-Id":self.client_id,
        "X-Naver-Client-Secret":self.client_secret
        }

        number_of_iter = (int(number) // 100) + 1

        crawl_result = []
        start = 1
        for i in range(number_of_iter):

            if i == (int(number) // 100):
                display = int(number) % 100
            else:
                display = 100
            

            data = {
                'query':query, # 검색어
                'display':display, # 검색 개수 default : 10, max : 100
                'start':start, # 검색 시작 위치 default : 1, max : 100
                # 'sort':date # 검색 결과 정렬 방법 : sim 정확도순, date 날짜순
            }
            
            # 100 단위로 떨어지는 number 값이 들어오면 마지막 iter때 display 값이 0이됨
            if display:
                data = urllib.parse.urlencode(data)
                result = requests.get("https://openapi.naver.com/v1/search/news.json?", headers=headers, params=data)

                crawl_result.extend(result.json()['items'])

            start += 100

        # 아 여기는 원래 본문 없음
        crawl_result = pd.DataFrame(crawl_result)
        return crawl_result

In [32]:
50 * 25000

1250000

In [24]:
from newspaper import Article

def context(x):
    try:
        article = Article(x, language='ko')
        article.download()
        article.parse()
        return article.text
    except:
        return None

In [25]:
import os
import sys

import pandas as pd
import numpy as np
#import requests
import multiprocessing as mp
from multiprocessing import freeze_support

from pandas_parallel_apply import DataFrameParallel

# from crawl import Crawl
# from context import context
import argparse


freeze_support()

# parser = argparse.ArgumentParser()
# parser.add_argument('--company_name', default='하이닉스')
# parser.add_argument('--sort', default='date')
# args = parser.parse_args()

print(' ### Api Call Start ###')
cw = Crawl(client_id = "TUWK9h7NsYO2gamKnZzO", client_secret = "kucd9CcnDn")
result = cw(query='하이닉스', number=1000)

print(' ### Parsering start ###')
dfp = DataFrameParallel(result, n_cores=16, pbar=True)
result['context'] = dfp['link'].apply(context)

result.to_pickle(f'./crawl_set/{args.company_name}_{args.sort}.pkl')

 ### Api Call Start ###
 ### Parsering start ###


100%|██████████| 125/125 [00:43<00:00,  2.88it/s]


NameError: name 'args' is not defined

In [14]:
cw

In [33]:
import pandas as pd
import numpy as np
import re

# import kss
from soynlp.normalizer import *
from collections import OrderedDict

from pykospacing import Spacing
from hanspell import spell_checker

from nltk import word_tokenize, sent_tokenize

from konlpy.tag import Mecab

import time
class Preprocess:
    def __init__(self):

        #self.spacing = Spacing()
        self.excluded_words = ["이하 뉴스1", "이 줄은 실제 뉴스"]
        self.mecab = Mecab()

    def _remove_bold(self, texts):

        texts = texts.replace('<b>','')
        texts = texts.replace('</b>','')

        return texts

    def _remove_url_email(self, texts):
        pattern_email = re.compile(r'[-_0-9a-z]+@[-_0-9a-z]+(?:\.[0-9a-z]+)+', flags=re.IGNORECASE)
        pattern_url = re.compile(r'(?:https?:\/\/)?[-_0-9a-z]+(?:\.[-_0-9a-z]+)+', flags=re.IGNORECASE)
        texts = pattern_email.sub('', texts)
        texts = pattern_url.sub('', texts)

        return texts

    def _remove_bracket(self, texts):

        pattern_bracket = re.compile(r'^((?:\[.+\])|(?:【.+】)|(?:<.+>)|(?:◆.+◆)\s)')
        texts = pattern_bracket.sub('', texts).strip()

        return texts

    def _split_context(self, texts):

        #context = texts.split("\n")

        context = sent_tokenize(texts)
        return context

    def _remove_html(self, texts):

        """
        HTML 태그를 제거합니다.
        ``<p>안녕하세요 ㅎㅎ </p>`` -> ``안녕하세요 ㅎㅎ ``
        """
        preprocessed_text = []
        for text in texts:
            text = re.sub(r"<[^>]+>\s+(?=<)|<[^>]+>", "", text).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_email(self, texts):
        """
        이메일을 제거합니다.
        ``홍길동 abc@gmail.com 연락주세요!`` -> ``홍길동  연락주세요!``
        """
        preprocessed_text = []
        for text in texts:
            text = re.sub(
                r"[a-zA-Z0-9+-_.]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "", text
            ).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_hashtag(self, texts):
        """
        해쉬태그(#)를 제거합니다.
        ``대박! #맛집 #JMT`` -> ``대박!  ``
        """
        preprocessed_text = []
        for text in texts:
            text = re.sub(r"#\S+", "", text).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_user_mention(self, texts):
        """
        유저에 대한 멘션(@) 태그를 제거합니다.
        ``@홍길동 감사합니다!`` -> `` 감사합니다!``
        """
        preprocessed_text = []
        for text in texts:
            text = re.sub(r"@\w+", "", text).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_url(self, texts):
        """
        URL을 제거합니다.
        ``주소: www.naver.com`` -> ``주소: ``
        """
        preprocessed_text = []
        for text in texts:
            text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "", text).strip()
            text = re.sub(r"pic\.(\w+\.)+\S*", "", text).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_bad_char(self, texts):
        """
        문제를 일으킬 수 있는 문자들을 제거합니다.
        """
        bad_chars = {"\u200b": "", "…": " ... ", "\ufeff": ""}
        preprcessed_text = []
        for text in texts:
            for bad_char in bad_chars:
                text = text.replace(bad_char, bad_chars[bad_char])
            text = re.sub(r"[\+á?\xc3\xa1]", "", text)
            if text:
                preprcessed_text.append(text)
        return preprcessed_text

    def _remove_press(self, texts):
        """
        언론 정보를 제거합니다.
        ``홍길동 기자 (연합뉴스)`` -> ````
        ``(이스탄불=연합뉴스) 하채림 특파원 -> ````
        """
        re_patterns = [
            r"\([^(]*?(뉴스|경제|일보|미디어|데일리|한겨례|타임즈|위키트리)\)",
            r"[가-힣]{0,5} (기자|선임기자|수습기자|특파원|객원기자|논설고문|통신원|연구소장) ",  # 이름 + 기자
            r"[가-힣]{1,}(뉴스|경제|일보|미디어|데일리|한겨례|타임|위키트리)",  # (... 연합뉴스) ..
            r"\(\s+\)",  # (  )
            r"\(=\s+\)",  # (=  )
            r"\(\s+=\)",  # (  =)
        ]

        preprocessed_text = []
        for text in texts:
            for re_pattern in re_patterns:
                text = re.sub(re_pattern, "", text).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_copyright(self, texts):
        """
        뉴스 내 포함된 저작권 관련 텍스트를 제거합니다.
        ``(사진=저작권자(c) 연합뉴스, 무단 전재-재배포 금지)`` -> ``(사진= 연합뉴스, 무단 전재-재배포 금지)`` TODO 수정할 것
        """
        re_patterns = [
            r"\<저작권자(\(c\)|ⓒ|©|\(Copyright\)|(\(c\))|(\(C\))).+?\>",
            r"저작권자\(c\)|ⓒ|©|(Copyright)|(\(c\))|(\(C\))",
        ]
        preprocessed_text = []
        for text in texts:
            for re_pattern in re_patterns:
                text = re.sub(re_pattern, "", text).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_photo_info(self, texts):
        """
        뉴스 내 포함된 이미지에 대한 label을 제거합니다.
        ``(사진= 연합뉴스, 무단 전재-재배포 금지)`` -> ````
        ``(출처=청주시)`` -> ````
        """
        preprocessed_text = []
        for text in texts:
            text = re.sub(
                r"\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ",
                "",
                text,
            ).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_useless_breacket(self, texts):
        """
        위키피디아 전처리를 위한 함수입니다.
        괄호 내부에 의미가 없는 정보를 제거합니다.
        아무런 정보를 포함하고 있지 않다면, 괄호를 통채로 제거합니다.
        ``수학(,)`` -> ``수학``
        ``수학(數學,) -> ``수학(數學)``
        """
        bracket_pattern = re.compile(r"\((.*?)\)")
        preprocessed_text = []
        for text in texts:
            modi_text = ""
            text = text.replace("()", "")  # 수학() -> 수학
            brackets = bracket_pattern.search(text)
            if not brackets:
                if text:
                    preprocessed_text.append(text)
                    continue
            replace_brackets = {}
            # key: 원본 문장에서 고쳐야하는 index, value: 고쳐져야 하는 값
            # e.g. {'2,8': '(數學)','34,37': ''}
            while brackets:
                index_key = str(brackets.start()) + "," + str(brackets.end())
                bracket = text[brackets.start() + 1 : brackets.end() - 1]
                infos = bracket.split(",")
                modi_infos = []
                for info in infos:
                    info = info.strip()
                    if len(info) > 0:
                        modi_infos.append(info)
                if len(modi_infos) > 0:
                    replace_brackets[index_key] = "(" + ", ".join(modi_infos) + ")"
                else:
                    replace_brackets[index_key] = ""
                brackets = bracket_pattern.search(text, brackets.start() + 1)
            end_index = 0
            for index_key in replace_brackets.keys():
                start_index = int(index_key.split(",")[0])
                modi_text += text[end_index:start_index]
                modi_text += replace_brackets[index_key]
                end_index = int(index_key.split(",")[1])
            modi_text += text[end_index:]
            modi_text = modi_text.strip()
            if modi_text:
                preprocessed_text.append(modi_text)
        return preprocessed_text

    def _remove_repeat_char(self, texts):
        preprocessed_text = []
        for text in texts:
            text = repeat_normalize(text, num_repeats=2).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _clean_punc(self, texts):
        punct_mapping = {
            "‘": "'",
            "₹": "e",
            "´": "'",
            "°": "",
            "€": "e",
            "™": "tm",
            "√": " sqrt ",
            "×": "x",
            "²": "2",
            "—": "-",
            "–": "-",
            "’": "'",
            "_": "-",
            "`": "'",
            "“": '"',
            "”": '"',
            "“": '"',
            "£": "e",
            "∞": "infinity",
            "θ": "theta",
            "÷": "/",
            "α": "alpha",
            "•": ".",
            "à": "a",
            "−": "-",
            "β": "beta",
            "∅": "",
            "³": "3",
            "π": "pi",
        }

        preprocessed_text = []
        for text in texts:
            for p in punct_mapping:
                text = text.replace(p, punct_mapping[p])
            text = text.strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_repeated_spacing(self, texts):
        """
        두 개 이상의 연속된 공백을 하나로 치환합니다.
        ``오늘은    날씨가   좋다.`` -> ``오늘은 날씨가 좋다.``
        """
        preprocessed_text = []
        for text in texts:
            text = re.sub(r"\s+", " ", text).strip()
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _remove_dup_sent(self, texts):
        """
        중복된 문장을 제거합니다.
        """
        texts = list(OrderedDict.fromkeys(texts))
        return texts

    def _spacing_sent(self, texts):
        """
        띄어쓰기를 보정합니다.
        """
        preprocessed_text = []
        for text in texts:
            text = self.spacing(text)
            if text:
                preprocessed_text.append(text)
        return preprocessed_text

    def _spell_check_sent(self, texts):
        """
        맞춤법을 보정합니다.
        """
        preprocessed_text = []
        for text in texts:
            try:
                spelled_sent = spell_checker.check(text)
                checked_sent = spelled_sent.checked
                if checked_sent:
                    preprocessed_text.append(checked_sent)
            except:
                preprocessed_text.append(text)
        return preprocessed_text

    def _excluded_word_filter(self, texts):
        """
        특정 단어를 포함하는 문장 필터링
        """
        preprocessed_text = []
        for text in texts:
            include_flag = False
            for word in self.excluded_words:
                if word in text:
                    include_flag = True
                    break
            if not include_flag:
                preprocessed_text.append(text)
        return preprocessed_text

    # def _additional_filter(self, texts):


    #     preprocessed_text = []
    #     for text in texts:

    def _morph_filter(self, texts):
        """
        명사(NN), 동사(V), 형용사(J)의 포함 여부에 따라 문장 필터링
        """
        NN_TAGS = ["NNG", "NNP", "NNB", "NP"]
        V_TAGS = ["VV", "VA", "VX", "VCP", "VCN", "XSN", "XSA", "XSV"]
        J_TAGS = ["JKS", "J", "JO", "JK", "JKC", "JKG", "JKB", "JKV", "JKQ", "JX", "JC", "JKI", "JKO", "JKM", "ETM"]

        preprocessed_text = []
        for text in texts:
            morphs = self.mecab.pos(text, join=False)

            nn_flag = False
            v_flag = False
            j_flag = False
            for morph in morphs:
                pos_tags = morph[1].split("+")
                for pos_tag in pos_tags:
                    if not nn_flag and pos_tag in NN_TAGS:
                        nn_flag = True
                    if not v_flag and pos_tag in V_TAGS:
                        v_flag = True
                    if not j_flag and pos_tag in J_TAGS:
                        j_flag = True
                if nn_flag and v_flag and j_flag:
                    preprocessed_text.append(text)
                    break
        return preprocessed_text

    def _remove_stopwords(self, sents):
        #  큰 의미가 없는 불용어 정의
        #  사용자가 자체적으로 stopwords를 정희하면 될 듯
        stopwords = ["소취요", "-", "조드윅", "포스터", "앓는", "서린"]
        preprocessed_text = []
        for sent in sents:
            sent = [w for w in sent.split(" ") if w not in stopwords]  # 불용어 제거
            preprocessed_text.append(" ".join(sent))
        return preprocessed_text

    def __call__(self, text):

        if text != None:

            context = text
            context = self._remove_bold(context)
            context = self._remove_url_email(context)
            context = self._remove_bracket(context)
            context = self._split_context(context)
            context = self._remove_html(context)
            #context = self._remove_email(context)
            context = self._remove_hashtag(context)
            context = self._remove_user_mention(context)
            #context = self._remove_url(context)
            context = self._remove_bad_char(context)
            context = self._remove_press(context)
            context = self._remove_copyright(context)
            context = self._remove_photo_info(context)
            context = self._remove_useless_breacket(context)
            context = self._remove_repeat_char(context)
            context = self._clean_punc(context)
            context = self._remove_repeated_spacing(context)
            context = self._remove_dup_sent(context)
            # #context = self._spacing_sent(context)
            # context = self._spell_check_sent(context)
            # context = self._excluded_word_filter(context)
            # context = self._remove_stopwords(context)
            # context = self._morph_filter(context)

            return context

        else:
            return None

if __name__ == '__main__':

    sample = pd.read_pickle('sample.pkl')
    prepro = Preprocess()

    start_time = time.time()
    result = prepro(sample.iloc[0])
    end_time = time.time()
    
    print(result)
    print('time check', end_time - start_time)

FileNotFoundError: [Errno 2] No such file or directory: 'sample.pkl'

In [40]:
result['title']

0      광주·전남 &quot;반도체 특화단지 여기가 적지입니다&quot;··· 장점 강조하...
1         [증시] 코스피, 개인·기관 매수에 소폭 상승…외인 순매도 상위 종목은 LG화...
2                        코스피, 개인·기관 매수에 소폭 상승...2450선 마감
3             [20일 마감시황] 코스피, 개인·기관 매수에 소폭 상승…2455.12 마감
4        美 정부 반도체 지원팀에 SK<b>하이닉스</b> 전 부사장 합류…역대급 로비 통했나
                             ...                        
995                       [정범구 칼럼] 예측 불가한 세계, 지속 가능한 지구?
996                               기재위서 막힌 반도체 설비투자 세액공제법
997                  용인 반도체클러스터 핵심기반시설 조성 &apos;순항&apos;
998               개미들 &apos;삼성전자 사랑&apos; 결실…주주 60%가 수익권
999       국회 &apos;K칩스법&apos; 논의 착수했지만…野 반대로 2월 통과는 가시밭길
Name: title, Length: 1000, dtype: object

In [42]:
prepro = Preprocess()

start_time = time.time()
result = prepro(result['title'][0])
end_time = time.time()

print(result)
print('time check', end_time - start_time)

['광주·전남 &quot;반도체 특화단지 여기가 적지입니다&quot;··· 장점 강조하며 구애']
time check 0.013570547103881836


In [46]:
"'가나"[0]

"'"

In [26]:
result

,title,originallink,link,description,pubDate,context
0,광주·전남 &quot;반도체 특화단지 여기가 적지입니다&quot;··· 장점 강조하...,http://www.mdilbo.com/detail/0kIA7d/689063,http://www.mdilbo.com/detail/0kIA7d/689063,전 <b>하이닉스</b> 반도체 총괄팀장을 단장으로 채용했다. 같은 달 반도체산업과...,"Mon, 20 Feb 2023 17:28:00 +0900","산업통상자원부는 반도체와 디스플레이, 이차전지를 국가 첨단전략산업으로 정하고 세부 ..."
1,"[증시] 코스피, 개인·기관 매수에 소폭 상승…외인 순매도 상위 종목은 LG화...",http://www.topstarnews.net/news/articleView.ht...,http://www.topstarnews.net/news/articleView.ht...,삼성전자(0.16%)와 SK<b>하이닉스</b>(0.54%) 등 반도체주는 소폭 올...,"Mon, 20 Feb 2023 17:28:00 +0900",[표] 외인 매수 매도 종목\n\n[표] 기관 매수 매도 종목\n\n[표] 코스피 ...
2,"코스피, 개인·기관 매수에 소폭 상승...2450선 마감",http://www.paxetv.com/news/articleView.html?id...,http://www.paxetv.com/news/articleView.html?id...,"코스피 시가총액 상위 10개 종목 중 NAVER(1.62%), 삼성바이오로직스(0....","Mon, 20 Feb 2023 17:20:00 +0900",코스피가 개인과 기관 순매수에 소폭 상승해 강보합권인 2450대에서 장을 마감했습니...
3,"[20일 마감시황] 코스피, 개인·기관 매수에 소폭 상승…2455.12 마감",http://www.newsworks.co.kr/news/articleView.ht...,http://www.newsworks.co.kr/news/articleView.ht...,코스피 시가총액 상위 10개 종목 중 ▲삼성전자(0.16%) ▲SK<b>하이닉스</...,"Mon, 20 Feb 2023 17:18:00 +0900",(사진=KRX 정보데이터시스템 캡처)\n\n[뉴스웍스=유한새 기자] 코스피는 개인과...
4,美 정부 반도체 지원팀에 SK<b>하이닉스</b> 전 부사장 합류…역대급 로비 통했나,https://biz.chosun.com/it-science/ict/2023/02/...,https://n.news.naver.com/mnews/article/366/000...,댄 김 전 SK<b>하이닉스</b> 미주 부사장 반도체팀 전략기획 책임자로 임명 삼...,"Mon, 20 Feb 2023 17:16:00 +0900",미국 정부가 반도체지원법(CHIPS and Science Act) 보조금 지원을 감...
...,...,...,...,...,...,...
995,"[정범구 칼럼] 예측 불가한 세계, 지속 가능한 지구?",http://www.mygoyang.com/news/articleView.html?...,http://www.mygoyang.com/news/articleView.html?...,[출처=EKOenergy] 당장 애플사는 협력기업들에게 2030년까지 RE100을 ...,"Tue, 14 Feb 2023 17:58:00 +0900","[고양신문] 2019년 말, 중국 우한에서 이름 모를 괴질로 사망자가 발생했을 때 ..."
996,기재위서 막힌 반도체 설비투자 세액공제법,http://www.dt.co.kr/contents.html?article_no=2...,https://n.news.naver.com/mnews/article/029/000...,장혜영 정의당 의원은 &quot;투자를 촉진한다는 법안 제도 취지는 좋지만 사실상 ...,"Tue, 14 Feb 2023 17:56:00 +0900",국회가 국내 반도체 산업 활성화를 위해 반도체 투자 세액공제율을 15%까지 상향하는...
997,용인 반도체클러스터 핵심기반시설 조성 &apos;순항&apos;,http://www.incheonilbo.com/news/articleView.ht...,http://www.incheonilbo.com/news/articleView.ht...,용인 반도체클러스터는 사업시행자인 용인일반산업단지(주)가 처인구 원삼면 일원에 약4...,"Tue, 14 Feb 2023 17:56:00 +0900",전력공급시설 공사 공정률 17%\n\n용수 공급 관로 설치도 본격화\n\n2027년...
998,개미들 &apos;삼성전자 사랑&apos; 결실…주주 60%가 수익권,https://www.sedaily.com/NewsView/29LQZ3OO7H,https://n.news.naver.com/mnews/article/011/000...,SK<b>하이닉스</b>(000660)에 대한 투자는 분위기가 사뭇 달랐다. 현재 ...,"Tue, 14 Feb 2023 17:54:00 +0900",삼성전자(005930)\n\nSK하이닉스(000660)\n\n[서울경제]‘국민주’가...


In [47]:
import pandas as pd

In [48]:
news1 = pd.read_excel('NewsResult_20220101-20230220.xlsx')
news2 = pd.read_excel('NewsResult_20200101-20211231.xlsx')
news3 = pd.read_excel('NewsResult_20180101-20191231.xlsx')
news4 = pd.read_excel('NewsResult_20150101-20171231.xlsx')

/opt/conda/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [53]:
news = pd.concat([news1, news2, news3, news4])

In [69]:
len(news)

58196

In [70]:
news = news[pd.isna(news['분석제외 여부'])]

In [72]:
news.head()

,뉴스 식별자,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건/사고 분류1,사건/사고 분류2,사건/사고 분류3,인물,위치,기관,키워드,특성추출(가중치순 상위 50개),본문,URL,분석제외 여부
0,7.101201e+06,20230220,디지털타임스,신하연,외국인 웃고 개미 울고 엇갈린 투자 성적표,경제>증권_증시,경제>외환,경제>부동산,NaN,NaN,NaN,김상인,"전주,미국","CJ제일제당,한국거래소,SK텔레콤,삼성전자,유가증권,SK하이닉스,삼성SDI,한국전력...","외국인,개미,투자,성적표,종목,외국인,투자자,매수,연초,수익,23.6%,수익률,기간...","외국인,반도체,수익률,코스피,에코프로비엠,연구원,가능성,투자자,sk,상승률,인플레이...",'연초 효과'가 이어지고 있지만 외국인과 개인의 투자 성적표는 극명히 갈리는 모습이...,http://www.dt.co.kr/contents.html?article_no=2...,NaN
1,8.100101e+06,20230220,KBS,신지수,“챗GPT 열풍을 기회로” 국내 반도체 산업 재도약할까,경제>반도체,경제>유통,NaN,NaN,NaN,NaN,"김우현,김재준",고성,"SK하이닉스,네이버,삼성전자,정부","열풍,기회로,재도약,반도체,산업,앵커,챗GPT,대화,인공,지능,돌풍,세계,기업들,A...","반도체,ai,차세대,부사장,콘퍼런스콜,삼성전자,고속연산,gpu,챗gpt,하이닉스,고...","[앵커]\n\n 대화형 인공지능 '챗GPT'의 돌풍 이후, 세계 주요 기업들이 AI...",https://news.kbs.co.kr/news/view.do?ncd=760948...,NaN
2,2.100501e+06,20230220,파이낸셜뉴스,김동호 기자 (hoya0222@fnnews.com),"치매 어르신 찾아주는 SK하이닉스, 기술에 온기를 담다",지역>대전,사회>장애인,지역>충남,NaN,NaN,NaN,"조모,곽노정","올림피아드,오학동,충남,하인슈타인,하인슈타인 프로젝트,여주시,경기도,이천,보령,청주...","보건복지부,독거노인종합지원,행복모아,푸르메재단,SPC삼립,하이닉스,SK하이닉스,장애...","치매,어르신,SK하이닉스,온기,지역,사회,행복,자발,기부,임직원,12년,온정,7만,...","장애인,하이닉스,sk하이닉스,사업장,발달장애인,배회감지기,지역사회,제과제빵,관계자,...",지역사회 행복 위해 자발적 기부 나선 임직원 \n12년간 297억 온정 모여 7만7...,http://www.fnnews.com/news/202302201911016525,NaN
3,2.100601e+06,20230220,한국경제,임현우,32% vs 12% 반도체ETF 수익률 천차만별,경제>반도체,경제>증권_증시,경제>유통,NaN,NaN,NaN,NaN,"대만,미국,미국필라델피아반도체나스닥,아시아,일본,중국,한국","ARIRANG,ETP전략팀,KBSTAR,SK하이닉스,SK하이닉스·SK스퀘어·,TIG...","12%,32%,vs,12%,반도체ETF,수익,천차,만별,엔비디아,반도체주,우상향,곡...","반도체,수익률,삼성전자,tiger,엔비디아,삼성증권,한국,etf,sk,하이닉스,연구...","엔비디아 49%, SK하이닉스 21%, TSMC 13% . \n \n올 들어 국내외...",https://www.hankyung.com/finance/article/20230...,NaN
4,1.600501e+06,20230220,무등일보,선정태,"광주 전남 ""반도체 특화단지 여기가 적지입니다"" 장점 강조하며 구애",지역>광주,지역>전남,지역>대전,NaN,NaN,NaN,"전태영,정성택,윤석열,김종갑","세종시,장성군,전남도,신룡지구,300만평,25만평,광주지역,태양,고려,광주시,제주시...","앰코테크놀로지,추진위원회,담양호,전남도,산업부,육성,하이닉스,인재양성위원회,전남대학...","광주,전남,반도체,특화,적지,장점,강조,구애,첨단,전략,산업,특화,단지,공모,마감,...","반도체,특화단지,광주,전남,지자체,전남도,반도체산업,만큼,전남대학교,설명회,하이닉스...",첨단전략산업 특화단지 공모 마감이 일주일 앞으로 다가오면서 광주 전남을 비롯한 전국...,http://www.mdilbo.com/detail/0kIA7d/689063,NaN


In [71]:
news.columns

Index(['뉴스 식별자', '일자', '언론사', '기고자', '제목', '통합 분류1', '통합 분류2', '통합 분류3',
       '사건/사고 분류1', '사건/사고 분류2', '사건/사고 분류3', '인물', '위치', '기관', '키워드',
       '특성추출(가중치순 상위 50개)', '본문', 'URL', '분석제외 여부'],
      dtype='object')

In [73]:
cols = ['일자', '제목', '통합 분류1', '통합 분류2', '통합 분류3', '키워드', '특성추출(가중치순 상위 50개)', '본문', 'URL']

In [81]:
news = news[cols].reset_index()

In [88]:
news = news.drop(columns=['index'])

In [90]:
news.to_pickle('하이닉스_news.pkl')